In [ ]:
#r "netstandard"
#load "Paket.fsx"

In [ ]:
// Paket.Dependencies.Install """
// frameworks: net472
// source https://nuget.org/api/v2
// nuget CNTK.CPUOnly
// nuget FsLab
// """

In [ ]:
Paket.Package [ "CNTK.CPUOnly"; "FsLab"]

In [ ]:
#load "Paket.Generated.Refs.fsx"

In [1]:
#r @"C:\Users\cernu\Documents\Code\IfCntk\packages\CNTK.CPUOnly\lib\netstandard2.0\Cntk.Core.Managed-2.6.dll"
//#r @"C:\Users\cernu\Documents\Code\IfCntk\bin\Cntk.Core.Managed-2.6.dll"
open System.Reflection
open System.Text.RegularExpressions
open System.IO
open System

let IsWindows =
    System.Runtime.InteropServices.RuntimeInformation.OSDescription.Contains("Windows")

let CntkCoreManagedPath =
    let codebase = Assembly.GetAssembly(typeof<CNTK.CNTKLib>).CodeBase 
    match IsWindows with
    | true -> codebase.Replace("file:///","") 
    | false -> codebase.Replace("file://","") 
    |> Path.GetDirectoryName

let CntkVersion =
    CntkCoreManagedPath
        .Split([|'/';'\\' |])
    |> Array.rev
    |> Array.skipWhile 
         (fun dir -> not <| Regex.IsMatch(dir, @"^\d\.\d(\.\d)?$"))
    |> function
    | [||] -> ""
    | dirs -> dirs |> Array.head
    |> fun str -> str.Replace("\\", "").Replace("/", "");

CntkCoreManagedPath, CntkVersion

("C:\Users\cernu\Documents\Code\IfCntk\bin", "")

In [ ]:
let PackagesPath =
    CntkCoreManagedPath
    |> Path.GetDirectoryName
    |> fun path -> path.Split([|'/';'\\' |])
    |> Array.rev
    |> Array.skipWhile 
        (fun dir -> not <| dir.ToLower().Contains("cntk."))
    |> Array.skip 1
    |> Array.rev
    |> Array.reduce 
        (fun state current -> sprintf "%s%s%s" state (Path.DirectorySeparatorChar.ToString()) current)

PackagesPath

In [ ]:
type DllKind = Debug | Release

let ManagedSupportPath kind =
    match kind with
    | Debug -> sprintf "../../support/x64/%s" "Debug"
    | Release -> sprintf "../../support/x64/%s" "Release"
    |> fun supportPath ->  
        Path.Combine(CntkCoreManagedPath,supportPath)
        |> Path.GetFullPath

ManagedSupportPath Debug

In [ ]:
let CudaPaths =
        Directory
            .GetDirectories(PackagesPath, "cntk.deps.cu*")
        |> Array.collect (fun dir -> Directory.GetDirectories(Path.Combine(dir, sprintf "%s/support/x64" CntkVersion)))
        |> Array.map Path.GetFullPath

CudaPaths

In [4]:
let PATH = "PATH"

let original_paths = Environment.GetEnvironmentVariable(PATH)

let GetCurrentPaths() =
    Environment
        .GetEnvironmentVariable(PATH)
        .Split([|Path.PathSeparator|], StringSplitOptions.RemoveEmptyEntries)

let AddPaths (paths :string []) = 
        
    paths 
    |> Array.tryFind (Directory.Exists >> not)
    |> function 
    | Some path -> 
        failwith <| sprintf "Folder '%s' not found, can't add to env:PATH" path        
    | _ -> 
        let current_paths = 
            GetCurrentPaths()
            |> Array.append paths
            |> Array.distinct

        Environment.SetEnvironmentVariable(
                PATH,
                System.String.Join(Path.PathSeparator.ToString(), current_paths))
                //EnvironmentVariableTarget.Process)

let AddPath path = AddPaths [|path|]

GetCurrentPaths() |> Array.sort

[|"C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\DAL";
  "C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\IPT";
  "C:\Program Files (x86)\Intel\Intel(R) Management Engine Components\iCLS\";
  "C:\Program Files (x86)\NVIDIA Corporation\PhysX\Common";
  "C:\Program Files\Git\cmd";
  "C:\Program Files\Intel\Intel(R) Management Engine Components\DAL";
  "C:\Program Files\Intel\Intel(R) Management Engine Components\IPT";
  "C:\Program Files\Intel\Intel(R) Management Engine Components\iCLS\";
  "C:\Program Files\Microsoft MPI\Bin\";
  "C:\Program Files\Microsoft SQL Server\130\Tools\Binn\";
  "C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v9.0\bin";
  "C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v9.0\libnvvp";
  "C:\Program Files\dotnet\"; "C:\Program Files\nodejs\";
  "C:\ProgramData\Boxstarter"; "C:\ProgramData\chocolatey\bin"; "C:\Python27\";
  "C:\Python27\Scripts"; "C:\Users\cernu\.dotnet\tools";
  "C:\Users\cernu\Anaconda3"; "C:\Users

In [ ]:
let DepPaths kind =
    let support = 
        Directory
            .GetDirectories(PackagesPath, "cntk.deps.*")
        |> Array.collect (fun dir -> 
            [|dir;CntkVersion;"support";"x64"|]
            |> Path.Combine
            |> Directory.GetDirectories)
        |> Array.map Path.GetFullPath
    
    let binPath = match kind with Debug -> "Debug" | Release -> "Release"
    let binPaths = 
        support 
        |> Array.map (fun dir -> Path.Combine(dir, binPath))
        |> Array.filter (Directory.Exists)
    support |> Array.append binPaths
    
DepPaths Debug

In [3]:
GetCurrentPaths()

The value or constructor 'GetCurrentPaths' is not defined.

In [ ]:
CntkCoreManagedPath |> AddPath
Release |> ManagedSupportPath |> AddPath
Release |> DepPaths |> AddPaths

GetCurrentPaths()

In [2]:
open CNTK
let cpu = DeviceDescriptor.UseDefaultDevice()
printfn "You are using CNTK for: %A" (cpu.Type)

You are using CNTK for: CPU


In [ ]:
System.Reflection.Assembly.GetExecutingAssembly().CodeBase

In [ ]:
let ResetPath() =
    GetCurrentPaths() 
    |> Array.filter (fun str -> str.ToLower().Contains("cntk") |> not) 
    |> Array.reduce(fun state current -> sprintf "%s%c%s" state (Path.PathSeparator) current)
    |> fun path -> Environment.SetEnvironmentVariable(PATH,path)

ResetPath()

In [ ]:
let binPaths kind = 
    seq {
        yield CntkCoreManagedPath
        match kind with
        | Debug -> 
            yield Debug |> ManagedSupportPath 
            yield! Debug |> DepPaths
        | Release ->
            yield Release |> ManagedSupportPath 
            yield! Release |> DepPaths
        
    } 

binPaths Release
|> Seq.map (fun path -> sprintf "#I \"%s\"" <| path.Replace("\\","/"))
|> Seq.iter (printfn "%s")

In [ ]:
#I "C:/Users/cernu/Documents/Code/IfSharp.v3.0.1/packages/CNTK.CPUOnly/lib/netstandard2.0"
#I "C:/Users/cernu/Documents/Code/IfSharp.v3.0.1/packages/CNTK.CPUOnly/support/x64/Debug"
#I "C:/Users/cernu/Documents/Code/IfSharp.v3.0.1/packages/CNTK.Deps.OpenCV.Zip/support/x64/Dependency/Debug"
#I "C:/Users/cernu/Documents/Code/IfSharp.v3.0.1/packages/CNTK.Deps.MKL/support/x64/Dependency"
#I "C:/Users/cernu/Documents/Code/IfSharp.v3.0.1/packages/CNTK.Deps.OpenCV.Zip/support/x64/Dependency"

In [ ]:
#I "C:/Users/cernu/Documents/Code/IfSharp.v3.0.1/packages/CNTK.CPUOnly/lib/netstandard2.0"
#I "C:/Users/cernu/Documents/Code/IfSharp.v3.0.1/packages/CNTK.CPUOnly/support/x64/Release"
#I "C:/Users/cernu/Documents/Code/IfSharp.v3.0.1/packages/CNTK.Deps.OpenCV.Zip/support/x64/Dependency/Release"
#I "C:/Users/cernu/Documents/Code/IfSharp.v3.0.1/packages/CNTK.Deps.MKL/support/x64/Dependency"
#I "C:/Users/cernu/Documents/Code/IfSharp.v3.0.1/packages/CNTK.Deps.OpenCV.Zip/support/x64/Dependency"

In [ ]:
if "bin" |> Directory.Exists |> not
then 
    Directory.CreateDirectory("bin") |> ignore
else 
    Directory.GetFiles("bin") 
    |> Array.iter(fun file -> printfn "Cleaning file '%s'..." file; FileInfo(file).Delete())
    
binPaths Release
|> Array.ofSeq
|> Array.collect (fun path -> Directory.GetFiles(path, "*.dll"))
|> Array.iter(fun path -> 
        printfn "Copying %s" path
        File.Copy(path, Path.Combine("bin", Path.GetFileName(path))))
        
"bin" |> Path.GetFullPath |> AddPath

In [ ]:
Directory.GetFiles("bin") 

In [ ]:
GetCurrentPaths()

In [ ]:
#I @"C:\Users\cernu\Documents\Code\IfCntk\bin"

In [ ]:
IntPtr.Size